In [1]:
import networkx as nx
import numpy as np
import itertools

In [2]:
a=np.load("UUU-top1000-5frag-4A.npz")
interactions = [a['interactions-%i'%i] for i in range(4)]

# Renomage, pour networkx
Sous la forme "ID_level" (ex. noeud 3 du niveau 0 : "3_0")

In [3]:
interactions_nn = []

In [4]:
for int_level in range(len(interactions)):
    curIntLevel = []
    for int_pair in range(interactions[int_level].shape[0]):
        curIntLevel.append([str(interactions[int_level][int_pair][0]) + "_" + str(int_level),
                            str(interactions[int_level][int_pair][1]) + "_" + str(int_level+1)])

    interactions_nn.append(curIntLevel)

# Création du graph

In [5]:
DG = nx.DiGraph()
for int_level in range(len(interactions_nn)):
    for edge in interactions_nn[int_level]:
        DG.add_edge(*edge)
DG.number_of_nodes(), DG.number_of_edges()

(1312, 7823)

In [6]:
unique_input_id = np.unique(interactions[0][:,0])
unique_output_id = np.unique(interactions[3][:,1])

unique_input_node = [i for i in DG.nodes() if "_0" in i]
unique_output_node = [i for i in DG.nodes() if "_4" in i]

# Exploration "dynamique"

Sans mémoire du meilleur parcours (Lent, ~500ms)

In [7]:
def countPath_old(node, DG):
    if len(list(DG.successors(node))) == 0:
        return 1
    else:
        return sum([countPath_old(p_node, DG) for p_node in DG.successors(node)])
    
#print(DG.successors(unique_input_node[0]), len(list(DG.successors(unique_input_node[0]))))
sum([countPath_old(start_node, DG) for start_node in unique_input_node])

565507

In [8]:
%%timeit -n 10 -r 10
sum([countPath_old(start_node, DG) for start_node in unique_input_node])

479 ms ± 7.18 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


Avec mémoire du meilleur parcours (Rapide, ~15ms)

In [9]:
def initCount(DG):
    for node in DG.nodes:
        DG.nodes[node]['count_path_value'] = -1

def countPath(node, DG):
    if DG.nodes[node]['count_path_value'] == -1:
        if len(list(DG.successors(node))) == 0:
            DG.nodes[node]['count_path_value']  = 1
        else:
            DG.nodes[node]['count_path_value']  = sum([countPath(p_node, DG) for p_node in DG.successors(node)])
    return DG.nodes[node]['count_path_value']

initCount(DG)
sum([countPath(start_node, DG) for start_node in unique_input_node])

565507

In [10]:
%%timeit -n 10 -r 10
initCount(DG)
sum([countPath(start_node, DG) for start_node in unique_input_node])

16.3 ms ± 944 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


# Bonus

In [13]:
def initPath(DG):
    for node in DG.nodes:
        DG.nodes[node]['bestSuccessor'] = None
        DG.nodes[node]['bestPath'] = []
        DG.nodes[node]['bestPath_value'] = 0
        DG.nodes[node]['node_value'] = int(node.split("_")[0])
        
def bestPath(node, DG):
    if DG.nodes[node]['bestSuccessor'] == None:
        if len(list(DG.successors(node))) == 0:
            DG.nodes[node]['bestSuccessor'] = (DG.nodes[node]['node_value'], node)
            DG.nodes[node]['bestPath_value'] = DG.nodes[node]['node_value']
        else:
            unsorted_successors = [[bestPath(p_node, DG)[0] + DG.nodes[node]['node_value'], p_node]
                                   for p_node in DG.successors(node)]
            sorted_successors = sorted(unsorted_successors, key=lambda tup: tup[0])
            DG.nodes[node]['bestSuccessor'] = sorted_successors[0]
            DG.nodes[node]['bestPath'] = [sorted_successors[0][1]] + DG.nodes[sorted_successors[0][1]]['bestPath']
            DG.nodes[node]['bestPath_value'] = DG.nodes[node]['node_value'] + DG.nodes[sorted_successors[0][1]]['bestPath_value']
    return DG.nodes[node]['bestSuccessor']

initPath(DG)
allBest = sorted([[bestPath(start_node, DG)[0] + DG.nodes[start_node]['node_value'], start_node]
                 for start_node in unique_input_node], 
                 key=lambda tup: tup[0])
[allBest[0][1]] + DG.nodes[allBest[0][1]]['bestPath'], DG.nodes[allBest[0][1]]['bestPath_value']

(['39_0', '101_1', '74_2', '98_3', '106_4'], 418)

In [14]:
%%timeit -r 10 -n 10
initPath(DG)
allBest = sorted([[bestPath(start_node, DG)[0] + DG.nodes[start_node]['node_value'], start_node]
                 for start_node in unique_input_node], 
                 key=lambda tup: tup[0])
[allBest[0][1]] + DG.nodes[allBest[0][1]]['bestPath'], DG.nodes[allBest[0][1]]['bestPath_value']

32.8 ms ± 2.12 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
